In [8]:
import os
import json
import openai
import time
from tqdm import tqdm
import polars as pl
from dotenv import dotenv_values

config = dotenv_values("../.env") 

In [18]:
def get_issue_reponses(title, body):
    cost = 0
    response = openai.ChatCompletion.create(
        engine=config["OPENAI_ENGINE"], 
        messages=[
            {"role": "system", "content": "You are a helpful assistant that is python master and have years of experience with github moderation."},
            {"role": "user", "content": f"Is the description of issue good? \ntitle: {title}\nbody: {body}\nAnswer with Yes/No as single word"},
        ]
    )
    description_quality = response['choices'][0]['message']['content']
    cost += response['usage']["total_tokens"]

    response = openai.ChatCompletion.create(
        engine=config["OPENAI_ENGINE"], 
        messages=[
            {"role": "system", "content": "You are a helpful assistant that is python master and have years of experience with github moderation."},
            {"role": "user", "content": f"Give me one sentence summary of following issue, use ten words or less: \ntitle: {title}\nbody: {body}"},
        ]
    )
    summary = response['choices'][0]['message']['content']
    cost += response['usage']["total_tokens"]

    response = openai.ChatCompletion.create(
        engine=config["OPENAI_ENGINE"], 
        messages=[
            {"role": "system", "content": "You are a helpful assistant that is python master and have years of experience with github moderation. You are maintainer of shap package"},
            {"role": "user", "content": f"Suggest short label for this issue. Label should be of maximum two words: \ntitle: {title}\nbody: {body}"},
        ]
    )
    short_label = response['choices'][0]['message']['content']
    cost += response['usage']["total_tokens"]

    issue_labels = [
        "bug",
        "documentation",
        "enhancement",
        "good first issue",
        "stale"
    ]
    response = openai.ChatCompletion.create(
        engine=config["OPENAI_ENGINE"], 
        messages=[
            {"role": "system", "content": "You are a helpful assistant that is python master and have years of experience with github moderation. You are maintainer of shap package"},
            {"role": "user", "content": f"Select label from {issue_labels}, where stale label is for issues that are too short and not clear: \ntitle: {title}\nbody: {body}\nAnswer with label name from list and nothing else"},
        ]
    )
    label = response['choices'][0]['message']['content']
    cost += response['usage']["total_tokens"]
    # response = openai.ChatCompletion.create(
    #     engine=config["OPENAI_ENGINE"], 
    #     messages=[
    #         {"role": "system", "content": "You are a helpful assistant that is python master and have years of experience with github moderation. You are maintainer of shap package"},
    #         {"role": "user", "content": f"Write first comment for: \ntitle: {title}\nbody: {body}\n"},
    #     ]
    # )
    # comment = response['choices'][0]['message']['content']

    return description_quality, summary, short_label, label, cost

In [21]:
openai.api_type = "azure"
openai.api_base = config["OPENAI_ENDPOINT"]
openai.api_version = "2023-05-15"
openai.api_key = config["OPENAI_KEY"]

data = pl.read_json("../data/issues_v1.json").filter(pl.col("number").is_not_null()).sort("number")
data = data.with_columns([
    pl.lit("").alias("description_quality"),
    pl.lit("").alias("summary"),
    pl.lit("").alias("short_label"),
    pl.lit("").alias("label"),
])

In [22]:
total_tokens = 0

In [28]:

for i in tqdm(range(len(data))):
    if data[i, "summary"] != "":
        continue
    row = data[i]
    title = row["title"]
    body = row["body"]
    try:
        description_quality, summary, short_label, label, tokens = get_issue_reponses(title, body)
        data[i, "description_quality"] = description_quality
        data[i, "summary"] = summary
        data[i, "short_label"] = short_label
        data[i, "label"] = label
        total_tokens += tokens
    except openai.error.RateLimitError:
        time.sleep(20)
        description_quality, summary, short_label, label, tokens = get_issue_reponses(title, body)
        data[i, "description_quality"] = description_quality
        data[i, "summary"] = summary
        data[i, "short_label"] = short_label
        data[i, "label"] = label
        total_tokens += tokens
    print(f"Total tokens: {total_tokens}")
    print(f"Average tokens per issue: {total_tokens / (i + 1)}")
    print(f"Estimated cost: {total_tokens / 1000 * 0.028}")

  3%|▎         | 88/2955 [00:02<01:14, 38.37it/s]

Total tokens: 38714
Average tokens per issue: 439.9318181818182
Estimated cost: 1.083992
Total tokens: 39141
Average tokens per issue: 439.7865168539326
Estimated cost: 1.095948


  3%|▎         | 90/2955 [00:27<19:56,  2.40it/s]

Total tokens: 39574
Average tokens per issue: 439.7111111111111
Estimated cost: 1.108072


  3%|▎         | 91/2955 [00:29<21:21,  2.24it/s]

...

Total tokens: 79843
Average tokens per issue: 438.6978021978022
Estimated cost: 2.235604


  6%|▌         | 183/2955 [19:34<8:10:49, 10.62s/it] 

Total tokens: 80284
Average tokens per issue: 438.7103825136612
Estimated cost: 2.247952


  6%|▌         | 183/2955 [19:41<4:58:23,  6.46s/it]


KeyboardInterrupt: 

In [29]:
data.write_json("../data/issues_v1_gpt.json", row_oriented=True, pretty=True)